In [6]:
import yfinance as yf
from backtesting import Backtest
import itertools
import os 

# 0. Config 
TICKER_LIST = ["AAPL", "MSFT", "GOOGL"]
START_DATE = "2019-01-01"
stats_to_keep = [
    "Start",
    "End",
    "Duration",
    "Equity Final [$]",
    "Return [%]",
    "Return (Ann.) [%]",
    "Volatility (Ann.) [%]",
    "Sharpe Ratio",
    "Sortino Ratio",
    "Max. Drawdown [%]",
    "# Trades",
    "Win Rate [%]",
    "Profit Factor",
    "Expectancy [%]"
]
hold_bar_list = [1, 5]

# 2. Create directories for results
path = os.getcwd()
html_results_path = os.path.dirname(path) + "/data/strategies/html_results/"
data_results_path = os.path.dirname(path) + "/data/strategies/csv_results/"
os.makedirs(html_results_path, exist_ok=True)
os.makedirs(data_results_path, exist_ok=True)

In [8]:
# Download data from Yahoo Finance
vxn_data = yf.download("^VXN", start=START_DATE, progress=False)[
    ["Open", "High", "Low", "Close", "Volume"]
].dropna().droplevel(1, axis=1)  # remove multi-index
vxn_data.index.name = "Date"

# Download data from FRED
import pandas as pd 
from fredapi import Fred
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))
import secret 

fred = Fred(api_key=secret.fred_api_key)

credit_spread_df = fred.get_series("BAMLH0A0HYM2").to_frame().dropna()
yield_curve_df = fred.get_series("T6MFF").to_frame().dropna()

credit_spread_df.index = pd.to_datetime(credit_spread_df.index)
yield_curve_df.index = pd.to_datetime(yield_curve_df.index)

# rename columns 0 as Close
credit_spread_df.columns = ["Close"]
yield_curve_df.columns = ["Close"]

# Only use after START_DATE
credit_spread_df = credit_spread_df[credit_spread_df.index >= START_DATE]
yield_curve_df = yield_curve_df[yield_curve_df.index >= START_DATE]

cs_index = credit_spread_df.index
yc_index = yield_curve_df.index
vxn_indx = vxn_data.index

In [ ]:
from backtesting import Backtest, Strategy
import pandas as pd
import numpy as np 

def RSI(values, n=14):
    """
    Calculate the Relative Strength Index for a given price series
    """
    # Calculate price changes
    delta = pd.Series(values).diff()
    # Split gains (up) and losses (down)
    up, down = delta.clip(lower=0), -delta.clip(upper=0)
    # Calculate EMA of up and down
    roll_up = up.rolling(n).mean()
    roll_down = down.rolling(n).mean()
    # Calculate RS and RSI
    rs = roll_up / roll_down
    rsi = 100.0 - (100.0 / (1.0 + rs))

    # Convert to numpy array with same length as input
    result = np.full_like(values, fill_value=np.nan)
    result[n:] = rsi.iloc[n:].values

    return rsi

class RSIStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5  # Par défaut comme dans l'exemple
    rsi_period = 14
    rsi_buy_threshold = 40
    # -----------------------------------

    def init(self):
        self.rsi = self.I(RSI, self.data.Close, self.rsi_period)

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Signal d'entrée : RSI < 40 (survendu)
        if self.rsi[-1] < self.rsi_buy_threshold:
            self.buy(size=1)

def BBANDS(values, n=20, dev=2):
    """
    Calculate Bollinger Bands for a price series
    Returns: middle band, upper band, lower band, and percent B
    """
    sma = pd.Series(values).rolling(n).mean()
    std = pd.Series(values).rolling(n).std(ddof=0)
    upper_band = sma + (std * dev)
    lower_band = sma - (std * dev)
    # Calculate %B (percent bandwidth)
    percent_b = (values - lower_band) / (upper_band - lower_band)
    
    return sma, upper_band, lower_band, percent_b

class BollingerStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    bb_period = 20
    bb_dev = 2
    # -----------------------------------

    def init(self):
        self.sma, self.upper, self.lower, self.percent_b = self.I(BBANDS, self.data.Close, self.bb_period, self.bb_dev)

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Acheter quand le prix est à l'intérieur des bandes (0 ≤ pctB ≤ 1)
        if 0 <= self.percent_b[-1] <= 1:
            self.buy(size=1)
        
        # Vendre quand le prix est au-dessus de la bande supérieure (pctB > 1)
        elif self.percent_b[-1] > 1:
            self.sell(size=1)

def CMF(high, low, close, volume, n=20):
    """
    Calculate Chaikin Money Flow
    """
    money_flow_multiplier = ((close - low) - (high - close)) / (high - low)
    money_flow_volume = money_flow_multiplier * volume
    money_flow_volume_series = pd.Series(money_flow_volume)
    volume_series = pd.Series(volume)
    cmf = money_flow_volume_series.rolling(n).sum() / volume_series.rolling(n).sum()
    # cmf = money_flow_volume.rolling(n).sum() / volume.rolling(n).sum()
    return cmf

class ChaikinMoneyFlowStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    cmf_period = 20
    cmf_buy_threshold = 0.20
    cmf_sell_threshold = -0.20
    # -----------------------------------

    def init(self):
        self.cmf = self.I(CMF, self.data.High, self.data.Low, self.data.Close, self.data.Volume)

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Acheter quand CMF > 0.20
        if self.cmf[-1] > self.cmf_buy_threshold:
            self.buy(size=1)
        
        # Vendre quand CMF < -0.20
        elif self.cmf[-1] < self.cmf_sell_threshold:
            self.sell(size=1)

def Stochastic_R(high, low, close, n=14):
    """
    Calculate Williams %R (similar to Lane Stochastic)
    """
    highest_high = pd.Series(high).rolling(n).max()
    lowest_low = pd.Series(low).rolling(n).min()
    percent_r = (highest_high - close) / (highest_high - lowest_low)
    return percent_r

class StochasticRStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    stoch_period = 14
    buy_threshold = 0.50
    sell_threshold = 0.90
    # -----------------------------------

    def init(self):
        self.percent_r = self.I(Stochastic_R, self.data.High, self.data.Low, self.data.Close, self.stoch_period)

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Acheter quand %R < 0.50
        if self.percent_r[-1] < self.buy_threshold:
            self.buy(size=1)
        
        # Vendre quand %R > 0.90
        elif self.percent_r[-1] > self.sell_threshold:
            self.sell(size=1)

def TDI_DI(values, n=14):
    """
    Calculate Trend Detection Index (TDI) and Direction Index (DI)
    """
    # Calcul simplifié - TDI est la pente du prix, DI est la dérivée seconde
    price = pd.Series(values)
    tdi = price.diff(n)  # Différence sur n périodes = momentum
    di = tdi.diff(1)     # Dérivée du momentum = accélération
    return tdi, di

class TDIStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    tdi_period = 14
    # -----------------------------------

    def init(self):
        self.tdi, self.di = self.I(TDI_DI, self.data.Close, self.tdi_period)

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Acheter quand TDI > 0 et DI > 0 (momentum croissant)
        if self.tdi[-1] > 0 and self.di[-1] > 0:
            self.buy(size=1)
        
        # Vendre quand TDI > 0 et DI < 0 (perte de momentum)
        elif self.tdi[-1] > 0 and self.di[-1] < 0:
            self.sell(size=1)

class Momentum60DayStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    # -----------------------------------

    def init(self):
        pass  # Pas besoin d'indicateur spécial ici

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        if len(self.data) > 60:
            if self.data.Close[-1] > self.data.Close[-61]:
                self.buy(size=1)

class VXNStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    vxn_period = 14
    vxn_buy_threshold = 0.20
    vxn_sell_threshold = -0.20
    # -----------------------------------

    def init(self):
        self.vxn_rsi = self.I(RSI, vxn_data.Close, self.vxn_period)

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Acheter quand VXN > 0.20
        if self.vxn_rsi[-1] < 30 or self.vxn_rsi[-1] > 80:
            self.buy(size=1)

class YieldCurveStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    yc_period = 14
    yc_buy_threshold = 0.20
    yc_sell_threshold = -0.20
    # -----------------------------------

    def init(self):
        self.yc_rsi= self.I(RSI, yield_curve_df.Close, self.yc_period)

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Acheter quand YC > 0.20
        if self.yc_rsi[-1] < 30 or self.yc_rsi[-1] > 80:
            self.buy(size=1)
    
class CreditSpreadStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    cs_period = 14
    cs_buy_threshold = 0.20
    cs_sell_threshold = -0.20
    # -----------------------------------

    def init(self):
        self.cs_rsi = self.I(RSI, credit_spread_df.Close, self.cs_period)

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Acheter quand CS > 0.20
        if self.cs_rsi[-1] < 30 or self.cs_rsi[-1] > 80:
            self.buy(size=1)

def williams_r(data, period=14):
    """
    Calcule l'indicateur Williams %R
    
    Args:
        data: DataFrame avec colonnes 'High', 'Low', 'Close'
        period: Période pour le calcul (défaut 14 jours)
    
    Returns:
        Series: Valeurs de Williams %R
    """
    highest_high = pd.Series(data['High']).rolling(window=period).max()
    lowest_low = pd.Series(data['Low']).rolling(window=period).min()
    # highest_high = data['High'].rolling(window=period).max()
    # lowest_low = data['Low'].rolling(window=period).min()
    williams = -100 * ((highest_high - data['Close']) / (highest_high - lowest_low))
    
    return williams

class WilliamsRStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    williams_period = 14
    williams_buy_threshold = -20
    williams_sell_threshold = -80
    # -----------------------------------

    def init(self):
        self.williams_r = self.I(williams_r, self.data, self.williams_period)

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Acheter quand Williams %R < -20
        if self.williams_r[-1] < self.williams_buy_threshold and self.williams_r[-1] > self.williams_sell_threshold:
            self.buy(size=1)
        
        # Vendre quand Williams %R > -80
        elif self.williams_r[-1] > self.williams_sell_threshold:
            self.sell(size=1)

In [12]:
strategy_list = [
    RSIStrategy,
    BollingerStrategy,
    ChaikinMoneyFlowStrategy,
    StochasticRStrategy,
    TDIStrategy,
    Momentum60DayStrategy,
    VXNStrategy,
    YieldCurveStrategy,
    CreditSpreadStrategy,
    WilliamsRStrategy,
]
# spread_data = yf.download("^VIX", start=START_DATE, progress=False)[
# results_df = pd.DataFrame(columns=["Ticker", "Strategy", "HoldBars"] + stats_to_keep)
results_df = pd.DataFrame()
for TICKER in TICKER_LIST:
    # 1. Download data
    data = yf.download(TICKER, start=START_DATE, progress=False)[
                ["Open", "High", "Low", "Close", "Volume"]
            ].dropna().droplevel(1, axis=1)  # remove multi-index
    data.index.name = "Date"
    
    # 3. Run backtests for each strategy and hold_bar combination
    for strategy, hold_bar in list(itertools.product(strategy_list, hold_bar_list)):

        if strategy == VXNStrategy:
            data = data[data.index.isin(vxn_indx)]
        elif strategy == YieldCurveStrategy:
            data = data[data.index.isin(yc_index)]
            yield_curve_df = yield_curve_df[yield_curve_df.index.isin(data.index)]
        elif strategy == CreditSpreadStrategy:
            data = data[data.index.isin(cs_index)]
            credit_spread_df = credit_spread_df[credit_spread_df.index.isin(data.index)]

        print(f"Running backtest with {TICKER} data...")
        print(f"Config : {strategy.__name__} with hold_bars={hold_bar}")

        strategy.hold_bars = hold_bar
        bt = Backtest(data, strategy,
                        cash=10_000, commission=0.002, trade_on_close=True)
        stats = bt.run()
        
        # Save results
        filename = f"{TICKER}_{strategy.__name__}_holdbars{hold_bar}"
        # Save plot
        bt.plot(plot_width=1200, 
                filename=html_results_path + filename + ".html",
                open_browser=False)
        # Save trades
        stats._trades.to_csv(data_results_path + filename + "_trades.csv", index=True)
        # Save stats
        stats_df = stats[stats_to_keep]
        stats_df.to_csv(data_results_path + filename + "_stats.csv", index=True)

        # Concat stats_df in results_df 
        # Save in a big dataframe 
        stats_df["Ticker"] = TICKER
        stats_df["Strategy"] = strategy.__name__
        stats_df["HoldBars"] = hold_bar
        
        results_df = pd.concat([results_df, stats_df], axis=1)



Running backtest with AAPL data...
Config : WilliamsRStrategy with hold_bars=1


Running backtest with AAPL data...
Config : WilliamsRStrategy with hold_bars=5


Running backtest with AAPL data...
Config : RSIStrategy with hold_bars=1


Running backtest with AAPL data...
Config : RSIStrategy with hold_bars=5


Running backtest with AAPL data...
Config : BollingerStrategy with hold_bars=1


...

Running backtest with GOOGL data...
Config : VXNStrategy with hold_bars=5


Running backtest with GOOGL data...
Config : YieldCurveStrategy with hold_bars=1


Running backtest with GOOGL data...
Config : YieldCurveStrategy with hold_bars=5


Running backtest with GOOGL data...
Config : CreditSpreadStrategy with hold_bars=1


Running backtest with GOOGL data...
Config : CreditSpreadStrategy with hold_bars=5


# Compiler resultat pour la moyenne

In [13]:
results_df = results_df.T
results_df

,Start,End,Duration,Equity Final [$],Return [%],Return (Ann.) [%],Volatility (Ann.) [%],Sharpe Ratio,Sortino Ratio,Max. Drawdown [%],# Trades,Win Rate [%],Profit Factor,Expectancy [%],Ticker,Strategy,HoldBars
0,2019-01-02 00:00:00,2025-04-30 00:00:00,2310 days 00:00:00,9141.550401,-8.584496,-1.411582,0.453733,-3.111039,-3.829507,-8.739597,1575,51.746032,1.054568,0.036892,AAPL,WilliamsRStrategy,1
0,2019-01-02 00:00:00,2025-04-30 00:00:00,2310 days 00:00:00,9489.832161,-5.101678,-0.825971,1.775552,-0.465191,-0.642659,-6.664834,1191,52.225021,1.083392,0.120557,AAPL,WilliamsRStrategy,5
0,2019-01-02 00:00:00,2025-04-30 00:00:00,2310 days 00:00:00,9875.39145,-1.246086,-0.198412,0.252019,-0.787288,-1.098079,-1.693951,327,51.376147,1.23681,0.186833,AAPL,RSIStrategy,1
0,2019-01-02 00:00:00,2025-04-30 00:00:00,2310 days 00:00:00,10023.39135,0.233914,0.037013,1.097535,0.033724,0.04857,-1.420859,327,51.070336,1.307996,0.480047,AAPL,RSIStrategy,5
0,2019-01-02 00:00:00,2025-04-30 00:00:00,2310 days 00:00:00,9372.184583,-6.278154,-1.021734,0.415693,-2.457905,-2.951674,-6.414094,1500,54.933333,1.218427,0.133133,AAPL,BollingerStrategy,1
0,2019-01-02 00:00:00,2025-04-30 00:00:00,2310 days 00:00:00,9983.614376,-0.163856,-0.025971,1.847386,-0.014058,-0.018962,-4.850044,1375,58.909091,1.486203,0.620133,AAPL,BollingerStrategy,5
0,2019-01-02 00:00:00,2025-04-30 00:00:00,2310 days 00:00:00,9811.492875,-1.885071,-0.300974,0.15709,-1.915938,-2.210889,-1.900612,296,48.648649,1.018575,0.00992,AAPL,ChaikinMoneyFlowStrategy,1
0,2019-01-02 00:00:00,2025-04-30 00:00:00,2310 days 00:00:00,9755.704035,-2.44296,-0.390981,0.687368,-0.568809,-0.733681,-3.35841,296,56.081081,1.132594,0.163499,AAPL,ChaikinMoneyFlowStrategy,5
0,2019-01-02 00:00:00,2025-04-30 00:00:00,2310 days 00:00:00,9486.028801,-5.139712,-0.832268,0.360595,-2.308041,-2.736227,-5.254993,1126,53.730018,1.148613,0.088987,AAPL,StochasticRStrategy,1
0,2019-01-02 00:00:00,2025-04-30 00:00:00,2310 days 00:00:00,9899.106808,-1.008932,-0.160488,1.418621,-0.11313,-0.152475,-4.561156,1077,61.188487,1.499561,0.585689,AAPL,StochasticRStrategy,5


In [14]:
for tic, hold_bar in itertools.product(TICKER_LIST, hold_bar_list):
    subdf = results_df[results_df["Ticker"] == tic]
    subdf = subdf[subdf["HoldBars"] == hold_bar]
    # drop Start, End, Duration, Equity Final [$], Ticker, Strategy, HoldBars
    subdf = subdf.drop(["Start", "End", "Duration", "Equity Final [$]", "Ticker", "Strategy", "HoldBars"], axis=1)
    # all columns to float 
    subdf = subdf.astype(float)
    # mean of all columns
    subdf = subdf.mean(axis=0)
    # save subdf 
    subdf = subdf.to_frame()

    subdf.to_csv(f"{data_results_path}{tic}_mean_holdbars{hold_bar}.csv",
                 index=True)
    # display(subdf)
